## Import Package

In [1]:
import random
import pickle
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Environment

In [3]:
class Environment():

    def __init__(
        self,

        # date range
        start_year,
        end_year,

        # name of two stocks
        x_stock_name,
        y_stock_name,

        # estimation info
        estimation_win,

        # trade info
        original_pv=1000,
        transaction_cost=0.001425

    ):
        # prepare testing data
        self.x_test, self.y_test = self.prepare_data(x_stock_name, y_stock_name, start_year, end_year)
        self.ratio_test = self.x_test / self.y_test
        print("Mean of Spread Ratio: ", np.mean(self.ratio_test))

        # estimation info
        self.estimation_win = estimation_win
        
        # trade info
        self.original_pv = original_pv
        self.transaction_cost = transaction_cost

    # prepare testing data
    def prepare_data(self, name1, name2, start_year, end_year):

        # intersect two stocks price on date
        x = pd.read_csv("/gdrive/MyDrive/專題/data/dataset1/{}.csv".format(name1))
        y = pd.read_csv("/gdrive/MyDrive/專題/data/dataset1/{}.csv".format(name2))
        x_y = pd.merge(left=x, right=y, how="inner", on="Date")

        # select part of dataframe in date range
        filter = x_y["Date"].str.contains(start_year)
        i = x_y[filter].index[0]
        filter = x_y["Date"].str.contains(end_year)
        j = x_y[filter].index[-1]
        x_y = x_y.iloc[i:j+1, :]

        # select date range
        x = np.array(x_y["Close_x"])
        y = np.array(x_y["Close_y"])

        return x, y

    # testing
    def testing(self):

        # info
        print("Number of day: ", len(self.x_test))

        # trade info initialization
        cash = self.original_pv
        x_count = 0
        y_count = 0

        # begin / end index
        begin_idx = self.estimation_win
        end_idx = len(self.ratio_test)
        current_idx = begin_idx

        # log history
        pv_history = []
        action_history = []
        spread_history = []

        # trading loop
        while current_idx != end_idx:

            # today info
            x_price = round(self.x_test[current_idx-1], 6)
            y_price = round(self.y_test[current_idx-1], 6)
            multiplier = round(self.ratio_test[current_idx-1], 6)
            spread_history.append(multiplier)

            # calculate pv
            pv = cash + (x_price * x_count) + (y_price * y_count)
            pv_history.append(pv)
            

            # get action
            temp_x = self.x_test[current_idx-self.estimation_win:current_idx]
            temp_y = self.y_test[current_idx-self.estimation_win:current_idx]
            temp_sp = temp_x / temp_y
            temp_sp = (temp_sp - np.mean(temp_sp)) / np.std(temp_sp)

            if temp_sp[-1] >= 1:
                action = 1
            elif temp_sp[-1] <= -1:
                action = 0
            else:
                action = 2

            action_history.append(action)
            

            # execute action
            # buy 1 x sell ratio y
            if action == 0:

                if x_count < 0 and y_count > 0:
                    cash += x_count * x_price
                    cash -= abs(x_count * x_price) * self.transaction_cost
                    x_count = 0

                    cash += y_count * y_price
                    cash -= y_count * y_price * self.transaction_cost
                    y_count = 0
                    

                # buy 1 x
                cash -= x_price
                cash -= x_price * self.transaction_cost 
                x_count += 1

                # sell ratio y
                cash += multiplier * y_price
                cash -= (multiplier * y_price) * self.transaction_cost
                y_count -= multiplier

            # sell 1 x buy ratio y
            elif action == 1:

                if x_count > 0 and y_count < 0:
                    cash += x_count * x_price
                    cash -= x_count * x_price * self.transaction_cost
                    x_count = 0

                    cash += y_count * y_price
                    cash -= abs(y_count * y_price) * self.transaction_cost
                    y_count = 0

                # sell 1 x
                cash += x_price
                cash -= x_price * self.transaction_cost
                x_count -= 1

                # buy ratio y
                cash -= multiplier * y_price
                cash -= (multiplier * y_price) * self.transaction_cost
                y_count += multiplier

            
            current_idx += 1
    

        return pv_history, action_history, spread_history

# Test

In [12]:
env = Environment(
    # date range
    start_year="2017",
    end_year="2017",

    # name of two stocks
    x_stock_name="MA",
    y_stock_name="CMCSA",

    # estimation / episode info
    estimation_win=120,

    # trade info
    original_pv=1000,
    transaction_cost=0.001425
)

Mean of Spread Ratio:  3.5301483357304857


In [13]:
pv_history, action_history, spread_history = env.testing()

Number of day:  251


In [14]:
def pv_sharpe_ratio(pv):
    changePerDay = []
    lastValue = 1
    for value in pv:
        changePerDay.append((value-lastValue)/lastValue)
        lastValue = value

    changePerDay.pop(0)
    avgValue = sum(changePerDay) / len(changePerDay)
    std = statistics.stdev(changePerDay)

    # convert into annual sharpe ration
    return (avgValue/std) * (len(pv) ** 0.5)

def pv_mdd(pv):
    arr = np.array(pv)
    argmin = arr.argmin()
    previousWin = arr[0:argmin]
    argmax = previousWin.argmax()
    return (pv[argmax] - pv[argmin]) / pv[argmax]

def pv_return(pv):
    return (pv[-1] - pv[0]) / pv[0]

In [15]:
pv_sharpe_ratio(pv_history)

1.1543497140049666

In [16]:
pv_mdd(pv_history)

2.441231250921115

In [17]:
pv_return(pv_history)

-0.6482945979558026

In [18]:
with open('rule.pickle', 'wb') as f:
    pickle.dump(pv_history, f)